In [14]:
import nltk
import requests
import pandas as pd
import numpy as np
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df = pd.read_csv('Processed_data.csv')

In [23]:
df.head()

,file,page,question,score,target
0,Question Paper - Unit 1 (WBS11) - January 2021...,2,Define the term ‘brand’ .,2,marketing mix and strategy
1,Question Paper - Unit 1 (WBS11) - January 2021...,2,Explain one risk Jack Ma may have taken when s...,4,entrepreneurs and leaders
2,Question Paper - Unit 1 (WBS11) - January 2021...,3,Analyse two factors that may have increased de...,6,the market
3,Question Paper - Unit 1 (WBS11) - January 2021...,4,Discuss if profit maximisation is the main bus...,8,entrepreneurs and leaders
4,Question Paper - Unit 1 (WBS11) - January 2021...,5,Assess the advantages of a paternalistic style...,10,managing people


In [24]:
df = df[~df['target'].isin(['unit 2', '?'])]

In [25]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    doc = nlp(' '.join(filtered_tokens))
    lemmatized_tokens = ' '.join([token.lemma_ for token in doc])
    return lemmatized_tokens

In [26]:
df['qst_processed'] = df['question'].apply(preprocess_text)
encoder = LabelEncoder()
y = encoder.fit_transform(df['target'])
df.drop(columns=['question', 'target', 'file'], inplace=True)
df['qst_len'] = df['qst_processed'].apply(len)
X = df.copy()

In [27]:
X

,page,score,qst_processed,qst_len
0,2,2,define term brand,17
1,2,4,explain one risk jack may take set alibaba group,48
2,3,6,analyse two factor may increase demand alibaba...,52
3,4,8,discuss profit maximisation main business obje...,65
4,5,10,assess advantage paternalistic style leadershi...,70
...,...,...,...,...
262,5,8,assess two factor could influence price elasti...,72
263,6,10,assess whether likely target young market segm...,62
266,9,4,explain flexible workforce might benefit ssp,44
267,10,4,construct supply demand diagram illustrate lik...,90


In [31]:
df.head().to_dict(orient='list')

dict_keys(['page', 'score', 'qst_processed', 'qst_len'])

In [39]:
url_set = 'http://0.0.0.0:8000/api/v1/models/set_model/'
payload = {
    "model_id": "log_reg"
}

response = requests.post(url_set, json=payload)

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response Body: {response.json()}")

Status Code: 422
Response Body: {'detail': [{'type': 'missing', 'loc': ['query', 'model_id'], 'msg': 'Field required', 'input': None}]}


In [51]:
url = 'http://0.0.0.0:8000/api/v1/models/fit/'
X = df.to_dict(orient='list')
data = {
    'X': X,
    'y' : y.tolist(),
}
response = requests.post(url, json=data)
if response.status_code == 201:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response: {'message': 'Model log_reg trained', 'data': {'roc_curve': {'0': {'fpr': [0.0, 0.0, 0.0, 0.29545454545454547, 0.30454545454545456, 1.0], 'tpr': [0.0, 0.047619047619047616, 1.0, 1.0, 1.0, 1.0], 'thresholds': [None, 4.120025355619882, 2.3512255374014916, -5.290704059019422, -5.293894371081741, -9.510708747123392]}, '1': {'fpr': [0.0, 0.0, 0.0, 0.9471153846153846, 0.9567307692307693, 1.0], 'tpr': [0.0, 0.030303030303030304, 1.0, 1.0, 1.0, 1.0], 'thresholds': [None, 5.33369414252594, 2.465768257778148, -7.506411036631318, -7.561297510690952, -8.991308218072007]}, '2': {'fpr': [0.0, 0.0, 0.0, 0.6956521739130435, 0.7065217391304348, 1.0], 'tpr': [0.0, 0.017543859649122806, 1.0, 1.0, 1.0, 1.0], 'thresholds': [None, 8.190081096396199, 2.2247963947552254, -5.493112045060312, -5.50674732055762, -8.456051065681143]}, '3': {'fpr': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0], 'tpr': [0.0, 0.010638297872340425, 0.13829787234042554, 0.1595744680851064, 1.0, 1.0], 'thresholds': [None, 8.60499708627155, 5

In [56]:
url_list = 'http://0.0.0.0:8000/api/v1/models/list_models/'
response = requests.get(url_list)
print(f"Status Code: {response.status_code}")
print(f"Response Body: {response.json()}")

Status Code: 200
Response Body: {'message': 'Models list', 'data': {'models_params': [{'log_reg': {'memory': 'None', 'steps': "[('tfidf', ColumnTransformer(remainder='passthrough',\n                  transformers=[('tfidf', TfidfVectorizer(stop_words='english'),\n                                 'qst_processed')])), ('model', LogisticRegression(C=103, max_iter=10000, multi_class='ovr'))]", 'verbose': 'False', 'tfidf': "ColumnTransformer(remainder='passthrough',\n                  transformers=[('tfidf', TfidfVectorizer(stop_words='english'),\n                                 'qst_processed')])", 'model': "LogisticRegression(C=103, max_iter=10000, multi_class='ovr')", 'tfidf__force_int_remainder_cols': 'True', 'tfidf__n_jobs': 'None', 'tfidf__remainder': 'passthrough', 'tfidf__sparse_threshold': '0.3', 'tfidf__transformer_weights': 'None', 'tfidf__transformers': "[('tfidf', TfidfVectorizer(stop_words='english'), 'qst_processed')]", 'tfidf__verbose': 'False', 'tfidf__verbose_feature_name

In [57]:
url = 'http://0.0.0.0:8000/api/v1/models/predict/'
X_test = df.to_dict(orient='list')
data = {
    'X': X_test,
}
response = requests.post(url, json=data)
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response: {'message': 'prediction with model log_reg done', 'data': {'model_id': 'log_reg', 'y_pred': [2, 0, 4, 0, 1, 3, 4, 2, 2, 1, 0, 3, 3, 2, 1, 1, 4, 2, 4, 3, 4, 2, 3, 2, 3, 4, 1, 2, 3, 3, 4, 4, 3, 2, 2, 1, 1, 2, 1, 3, 1, 3, 3, 1, 1, 3, 2, 3, 3, 3, 3, 0, 4, 0, 3, 0, 3, 3, 3, 2, 1, 3, 4, 0, 3, 2, 3, 1, 2, 2, 4, 3, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 4, 2, 2, 0, 3, 4, 1, 3, 2, 4, 1, 2, 3, 1, 4, 3, 3, 3, 3, 2, 3, 3, 0, 2, 2, 2, 3, 4, 2, 3, 0, 1, 3, 3, 2, 4, 1, 1, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 4, 2, 1, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 3, 2, 0, 2, 2, 3, 1, 3, 2, 3, 2, 4, 3, 2, 1, 3, 3, 4, 4, 1, 2, 1, 0, 2, 1, 2, 4, 3, 3, 4, 4, 0, 3, 1, 2, 3, 2, 3, 3, 4, 2, 3, 3, 2, 3, 0, 3, 4, 3, 1, 3, 3, 4, 0, 3, 2, 3, 4, 3, 3, 2, 3, 3, 2, 4, 3, 2, 0, 0, 4, 4, 3, 3, 3, 3, 2, 0, 0, 2, 3, 4, 4, 2, 2, 3, 2, 3, 4, 3, 1, 4, 1], 'y_pred_proba': [[0.02555314430536053, 0.0007577235740265563, 0.9251734498477109, 0.03623433175490826, 0.012281350517993846], [0.9500928948970706, 0.013036025739900543, 0.01

In [47]:
X_test

{'page': [2,
  2,
  3,
  4,
  5,
  7,
  7,
  8,
  9,
  10,
  3,
  4,
  5,
  6,
  11,
  11,
  12,
  13,
  14,
  2,
  2,
  3,
  4,
  5,
  7,
  7,
  8,
  9,
  10,
  2,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  11,
  3,
  3,
  4,
  5,
  6,
  8,
  11,
  11,
  12,
  13,
  14,
  16,
  2,
  2,
  3,
  4,
  5,
  7,
  7,
  8,
  9,
  10,
  10,
  10,
  11,
  12,
  15,
  16,
  17,
  18,
  19,
  5,
  6,
  7,
  9,
  18,
  24,
  4,
  5,
  6,
  8,
  10,
  15,
  16,
  21,
  2,
  2,
  3,
  4,
  6,
  8,
  8,
  9,
  10,
  12,
  2,
  3,
  4,
  6,
  9,
  13,
  14,
  20,
  2,
  2,
  3,
  4,
  6,
  8,
  8,
  9,
  10,
  12,
  4,
  5,
  6,
  8,
  10,
  15,
  16,
  2,
  2,
  3,
  4,
  5,
  7,
  7,
  8,
  9,
  10,
  5,
  6,
  9,
  11,
  16,
  17,
  20,
  22,
  3,
  3,
  4,
  5,
  6,
  8,
  11,
  11,
  12,
  13,
  14,
  16,
  3,
  3,
  4,
  5,
  6,
  8,
  11,
  11,
  12,
  13,
  14,
  2,
  2,
  3,
  4,
  5,
  7,
  7,
  8,
  9,
  10,
  4,
  5,
  6,
  8,
  10,
  15,
  17,
  21,
  2,
  3,
  4,
  5,
  6,
  8,
  8,
  9,